In [1]:
import pandas as pd
import os
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as krs
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm.autonotebook import tqdm
from tqdm import trange

/Users/filipwojcik/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  del sys.path[0]


In [2]:
%matplotlib inline

In [3]:
%load_ext autoreload

%autoreload 2

In [4]:
if os.getcwd().endswith("notebooks"):
    os.chdir(os.pardir)

In [5]:
sys.path.append(os.getcwd())

In [6]:
from src.data.train_data_utils import prepare_experiment_data
from src.models.training_utils import generator_ratings, generator_ratings_features, mae_masked, mape_masked, mse_masked
from src.models.train_model import get_collabfiltering_model1, build_shallow_autorec_single_input, build_autorec_multi_input2, get_array

In [7]:
tf.__version__

'2.0.0'

# Data prep

In [9]:
exd = prepare_experiment_data()

Processed: 0.0%
Processed: 0.27373261797875836%
Processed: 0.5474652359575167%
Processed: 0.8211978539362751%
Processed: 0.0%
Processed: 0.4555393586005831%
Processed: 0.9110787172011662%
Processed: 0.0%
Processed: 0.6804109682248077%


In [10]:
ytest = np.expand_dims(exd.Xraw_test.rating.to_numpy(), axis=1)

# Model prep

## Collaborative filtering with bias

In [14]:
dim_embed = 30
nusers = max(exd.Xraw_train.user_id.max(), exd.Xraw_test.user_id.max(), exd.Xraw_valid.user_id.max())
nitems = exd.Xraw_train.item_id.max()

user_input = krs.Input(shape=(1,), name="user_input", dtype='int32')
u = krs.layers.Embedding(nusers + 1, dim_embed)(user_input)
u = krs.layers.Flatten()(u)
u_bias = tf.keras.layers.Flatten()(krs.layers.Embedding(nusers + 1, 1)(user_input))

item_input = krs.Input(shape=(1,), name='item_input', dtype='int32')
i = krs.layers.Embedding(nitems + 1, dim_embed)(item_input)
i = krs.layers.Flatten()(i)
i_bias = krs.layers.Flatten()(krs.layers.Embedding(nitems + 1, 1)(item_input))

ui = krs.layers.Dot(axes=(1,1))([u, i])

all_features = krs.layers.Concatenate()([ui, u_bias, i_bias])
drop1 = krs.layers.Dropout(0.4)(all_features)
out = krs.layers.Dense(1)(drop1)

model_mf = krs.Model(inputs=[user_input, item_input], outputs=out)
model_mf.compile(loss=mse_masked, optimizer='adam', metrics=[mse_masked, 'mae', mape_masked])

In [15]:
model_mf.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 30)        610050      user_input[0][0]                 
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 30)        196680      item_input[0][0]                 
____________________________________________________________________________________________

In [56]:
x_mf = [
        exd.Xraw_train.user_id.to_numpy(),
        exd.Xraw_train.item_id.to_numpy()
    ]
y_mf = exd.Xraw_train.rating.to_numpy()

x_val_mf = [
    exd.Xraw_valid.user_id.to_numpy(),
    exd.Xraw_valid.item_id.to_numpy()
]
y_val_mf = exd.Xraw_valid.rating.to_numpy()



In [57]:
modelcf_history = model_mf.fit(x=x_mf, y=y_mf, batch_size=32, epochs=10, validation_data=(x_val_mf, y_val_mf))

Train on 36532 samples, validate on 14697 samples
Epoch 1/10
36532/36532 [==============================] - 18s 498us/sample - loss: 14.1320 - mse_masked: 14.1303 - mae: 3.5450 - mape_masked: 0.8051 - val_loss: 9.5989 - val_mse_masked: 9.5940 - val_mae: 2.8994 - val_mape_masked: 0.6140
Epoch 2/10
36532/36532 [==============================] - 16s 435us/sample - loss: 6.0479 - mse_masked: 6.0473 - mae: 2.2049 - mape_masked: 0.3765 - val_loss: 4.3561 - val_mse_masked: 4.3541 - val_mae: 1.9120 - val_mape_masked: 0.2845
Epoch 3/10
36532/36532 [==============================] - 16s 439us/sample - loss: 2.6714 - mse_masked: 2.6711 - mae: 1.3849 - mape_masked: 0.0704 - val_loss: 2.8911 - val_mse_masked: 2.8903 - val_mae: 1.5617 - val_mape_masked: 0.1470
Epoch 4/10
36532/36532 [==============================] - 14s 384us/sample - loss: 1.6099 - mse_masked: 1.6099 - mae: 1.0404 - mape_masked: -0.0452 - val_loss: 2.4357 - val_mse_masked: 2.4356 - val_mae: 1.4375 - val_mape_masked: 0.0932
Epoch 5

In [59]:
yhat_mf = model_mf.predict(x=[
    exd.Xraw_test.user_id.to_numpy(dtype='int32'),
    exd.Xraw_test.item_id.to_numpy(dtype='int32')
])

In [62]:
mape_masked(ytest, yhat_mf).numpy()

-0.122368895

## Deep recommender

In [9]:
from sklearn.preprocessing import StandardScaler

ytrain_mu = np.mean(exd.Xraw_train.rating)
ytrain_norm = (exd.Xraw_train.rating - ytrain_mu).to_numpy()
yval_norm = (exd.Xraw_valid.rating - ytrain_mu).to_numpy()
ytest_norm = (exd.Xraw_test.rating - ytrain_mu).to_numpy()

In [13]:
exd.Xfeatures_train.shape

(36532, 57)

In [77]:
dim_embed = 30
nusers = max(exd.Xraw_train.user_id.max(), exd.Xraw_test.user_id.max(), exd.Xraw_valid.user_id.max())
nitems = exd.Xraw_train.item_id.max()

f_input = krs.Input(shape=(exd.Xfeatures_train.shape[1],), name="features_input", dtype='float32')
f_drop = krs.layers.Dropout(rate=0.5)(f_input)
f_dense = krs.layers.Dense(16)(f_drop)


user_input = krs.Input(shape=(1,), name="user_input", dtype='int32')
u = krs.layers.Embedding(nusers + 1, dim_embed)(user_input)
u = krs.layers.Flatten()(u)
u_bias = tf.keras.layers.Flatten()(krs.layers.Embedding(nusers + 1, 1)(user_input))

item_input = krs.Input(shape=(1,), name='item_input', dtype='int32')
i = krs.layers.Embedding(nitems + 1, dim_embed)(item_input)
i = krs.layers.Flatten()(i)
i_bias = krs.layers.Flatten()(krs.layers.Embedding(nitems + 1, 1)(item_input))

ui = krs.layers.Dot(axes=(1,1))([u, i])
#ui = krs.layers.Multiply()([u, i])

all_features = krs.layers.Concatenate(axis=1)([ui, f_dense]) #u_bias, i_bias])
drop =  krs.layers.Dropout(rate=0.2)(all_features)
d1 = krs.layers.Dense(32, activation="relu")(drop)
drop2 = krs.layers.Dropout(rate=0.2)(d1)
d2 = krs.layers.Dense(16, activation="relu")(drop2)
out = krs.layers.Dense(1, )(d2)

model = krs.Model(inputs=[user_input, item_input, f_input], outputs=out )
model.compile(loss=mse_masked, optimizer='adam', metrics=[mse_masked, 'mae', mape_masked])

In [78]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 30)        610050      user_input[0][0]                 
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 30)        196680      item_input[0][0]                 
______________________________________________________________________________________________

In [79]:
x = [
        exd.Xraw_train.user_id.to_numpy(),
        exd.Xraw_train.item_id.to_numpy(),
        exd.Xfeatures_train
    ]
y = exd.Xraw_train.rating.to_numpy()

x_val = [
    exd.Xraw_valid.user_id.to_numpy(),
    exd.Xraw_valid.item_id.to_numpy(),
    exd.Xfeatures_valid
]
y_val = exd.Xraw_valid.rating.to_numpy()

In [80]:
es = krs.callbacks.EarlyStopping(monitor='val_mape_masked', mode='min', min_delta=1, patience=3)
model_history = model.fit(x=x, y=y, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Train on 36532 samples, validate on 14697 samples
Epoch 1/10
36532/36532 [==============================] - 18s 495us/sample - loss: 2.2149 - mse_masked: 2.2147 - mae: 1.1521 - mape_masked: -0.1483 - val_loss: 1.4280 - val_mse_masked: 1.4299 - val_mae: 0.9759 - val_mape_masked: -0.1429
Epoch 2/10
36532/36532 [==============================] - 19s 522us/sample - loss: 1.5010 - mse_masked: 1.5008 - mae: 0.9613 - mape_masked: -0.2063 - val_loss: 1.3887 - val_mse_masked: 1.3910 - val_mae: 0.9467 - val_mape_masked: -0.1779
Epoch 3/10
36532/36532 [==============================] - 16s 425us/sample - loss: 1.1556 - mse_masked: 1.1554 - mae: 0.8056 - mape_masked: -0.1800 - val_loss: 1.4208 - val_mse_masked: 1.4232 - val_mae: 0.9705 - val_mape_masked: -0.1588
Epoch 4/10
36532/36532 [==============================] - 15s 408us/sample - loss: 0.6977 - mse_masked: 0.6977 - mae: 0.5884 - mape_masked: -0.1142 - val_loss: 1.4027 - val_mse_masked: 1.4058 - val_mae: 0.9316 - val_mape_masked: -0.2036
Ep

In [81]:
yhat = model.predict(x=[
    exd.Xraw_test.user_id.to_numpy(dtype='int32'),
    exd.Xraw_test.item_id.to_numpy(dtype='int32'),
    exd.Xfeatures_test.astype('float32')
])
#yhat_denorm = yhat + ytrain_mu

In [82]:
mape_masked(ytest, yhat).numpy() #yhat_denorm).numpy()

-0.23286569

In [76]:
import gc
tf.keras.backend.clear_session()
gc.collect()

70545